In [28]:
"""a model developed for the first task is then reused as a starting point for another task"""

'a model developed for the first task is then reused as a starting point for another task'

In [29]:
import torch
import torch.nn as nn
import numpy as np
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as f
import matplotlib.pyplot as plt

In [30]:
import os
import time
import copy
from torchvision import datasets,transforms,models

In [31]:
device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [32]:
mean=np.array([0.485,0.456,0.406])
std=np.array([0.229,0.224,0.225])

In [33]:
data_transforms={
    'train':transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean,std)
    ]),  
    'val':transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean,std)
    ])
}

In [34]:
data_dir='hymenoptera_data'
sets=['train','val']
image_datasets={x:datasets.ImageFolder(os.path.join(data_dir,x),data_transforms[x]) for x in ['train','val']}

In [35]:
dataloaders={x:torch.utils.data.DataLoader(image_datasets[x],batch_size=4,shuffle=True,num_workers=0)
                for x in ['train','val']}

In [36]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
print(class_names)

['ants', 'bees']


In [37]:
def imshow(inp, title):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    plt.title(title)
    plt.show()

In [38]:
# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

In [39]:
def train_model(model,criterion,optimizer,scheduler,num_epochs=25):
    since=time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [40]:
#### Finetuning the convnet ####
# Load a pretrained model and reset final fully connected layer.

model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\user/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100.0%


In [41]:
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model.fc = nn.Linear(num_ftrs, 2)

model = model.to(device)

In [42]:
import torch.optim as optim

In [43]:
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [44]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [45]:
# StepLR Decays the learning rate of each parameter group by gamma every step_size epochs
# Decay LR by a factor of 0.1 every 7 epochs
# Learning rate scheduling should be applied after optimizer’s update
# e.g., you should write your code this way:
# for epoch in range(100):
#     train(...)
#     validate(...)
#     scheduler.step()

step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 0.6518 Acc: 0.6107
val Loss: 0.4793 Acc: 0.7778

Epoch 1/24
----------
train Loss: 0.5472 Acc: 0.7049
val Loss: 0.3683 Acc: 0.8758

Epoch 2/24
----------
train Loss: 0.4796 Acc: 0.7500
val Loss: 0.3173 Acc: 0.9216

Epoch 3/24
----------
train Loss: 0.4249 Acc: 0.8279
val Loss: 0.2814 Acc: 0.9216

Epoch 4/24
----------
train Loss: 0.3625 Acc: 0.8730
val Loss: 0.2498 Acc: 0.9216

Epoch 5/24
----------
train Loss: 0.3428 Acc: 0.8648
val Loss: 0.2361 Acc: 0.9216

Epoch 6/24
----------
train Loss: 0.4234 Acc: 0.8033
val Loss: 0.2209 Acc: 0.9346

Epoch 7/24
----------
train Loss: 0.3692 Acc: 0.8525
val Loss: 0.2246 Acc: 0.9346

Epoch 8/24
----------
train Loss: 0.2862 Acc: 0.8893
val Loss: 0.2342 Acc: 0.9281

Epoch 9/24
----------
train Loss: 0.3149 Acc: 0.8811
val Loss: 0.2196 Acc: 0.9281

Epoch 10/24
----------
train Loss: 0.3257 Acc: 0.8566
val Loss: 0.2271 Acc: 0.9281

Epoch 11/24
----------
train Loss: 0.3240 Acc: 0.8607
val Loss: 0.2212 Acc: 0.9281

Ep

In [46]:
#### ConvNet as fixed feature extractor ####
# Here, we need to freeze all the network except the final layer.
# We need to set requires_grad == False to freeze the parameters so that the gradients are not computed in backward()
model_conv = torchvision.models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 0.6577 Acc: 0.7049
val Loss: 0.2588 Acc: 0.8954

Epoch 1/24
----------
train Loss: 0.5727 Acc: 0.7787
val Loss: 0.1817 Acc: 0.9346

Epoch 2/24
----------
train Loss: 0.4868 Acc: 0.7500
val Loss: 0.1932 Acc: 0.9216

Epoch 3/24
----------
train Loss: 0.4670 Acc: 0.8033
val Loss: 0.2844 Acc: 0.9020

Epoch 4/24
----------
train Loss: 0.9397 Acc: 0.7008
val Loss: 0.4595 Acc: 0.8301

Epoch 5/24
----------
train Loss: 0.5082 Acc: 0.8074
val Loss: 0.2203 Acc: 0.9346

Epoch 6/24
----------
train Loss: 0.5043 Acc: 0.8074
val Loss: 0.2214 Acc: 0.9346

Epoch 7/24
----------
train Loss: 0.3115 Acc: 0.8525
val Loss: 0.2049 Acc: 0.9412

Epoch 8/24
----------
train Loss: 0.3152 Acc: 0.8730
val Loss: 0.2184 Acc: 0.9412

Epoch 9/24
----------
train Loss: 0.3313 Acc: 0.8361
val Loss: 0.1920 Acc: 0.9608

Epoch 10/24
----------
train Loss: 0.4280 Acc: 0.7951
val Loss: 0.1971 Acc: 0.9412

Epoch 11/24
----------
train Loss: 0.3598 Acc: 0.8525
val Loss: 0.1938 Acc: 0.9477

Ep